In [1]:
import os, subprocess, re

## Convert SPARQL PP into N3 PP

In [2]:
from rdflib.plugins.sparql import parser
from convert_pp_sparql_n3 import To_N3_Visitor

In [ ]:
# convert single query

query = "SELECT ?x WHERE { ?x (:p1/:p2)* ?z ; !(:p3|:p4|:p5) ?a }"
query = "PREFIX : <http://example.org/gmark/> " + \
    "SELECT * WHERE { ?x0 ((^:p1/:p2*)?/:p3)+ ?x3 } "
    # "SELECT * WHERE { ?x0 !(:p1|^:p2|:p3) ?x3 }"

query = parser.parseQuery(query)
query = query[1]

print(To_N3_Visitor().convert(query))

In [9]:

# convert query folder

import os

visitor = To_N3_Visitor()

path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    print(file)
    with open(os.path.join(path, file), 'r') as fh:
        query = fh.read()
        query = parser.parseQuery(query)
        query = query[1]
        
        conv = visitor.convert(query)
        conv = "@prefix : <http://example.org/gmark/> .\n\n" + conv
        # print(conv)
        
        n3_file = file[0:file.index(".")] + ".n3"
        with open(os.path.join(path, "n3", n3_file), 'w') as fh2:
            fh2.write(conv)
            
        # print("\n")

query-0.sparql
query-1.sparql
query-10.sparql
query-11.sparql
query-12.sparql
query-13.sparql
query-14.sparql
query-15.sparql
query-16.sparql
query-17.sparql
query-18.sparql
query-19.sparql
query-2.sparql
query-20.sparql
query-21.sparql
query-22.sparql
query-23.sparql
query-24.sparql
query-25.sparql
query-26.sparql
query-27.sparql
query-28.sparql
query-29.sparql
query-3.sparql
query-30.sparql
query-31.sparql
query-32.sparql
query-33.sparql
query-34.sparql
query-35.sparql
query-36.sparql
query-37.sparql
query-38.sparql
query-39.sparql
query-4.sparql
query-40.sparql
query-41.sparql
query-42.sparql
query-43.sparql
query-44.sparql
query-45.sparql
query-46.sparql
query-47.sparql
query-48.sparql
query-49.sparql
query-5.sparql
query-6.sparql
query-7.sparql
query-8.sparql
query-9.sparql


## Run SPARQL PP

In [ ]:
from resCSV2NT import convert as csv2nt

In [ ]:
def exec_sparql(query_file, data_file, result_file):
    process = subprocess.Popen(['java', '-jar', "../test/run/sparql.jar", "-n3", data_file, "-query", query_file], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    print(out)
    print(error)
    
    with open(result_file, 'w') as fh:
        fh.write(out.strip())

In [ ]:
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix"
files = list(os.listdir(path))
files.sort()
for file in files:
    if not file.endswith(".sparql"):
        continue
    # print(file)
    
    name = file[:file.index(".")]
    query_file = os.path.join(path, file)
    data_file = os.path.join(path, "data.n3")
    result_file_csv = os.path.join(path, "results", f"{name}.csv")
    result_file_nt = os.path.join(path, "results", f"{name}.nt")
    
    exec_sparql(query_file, data_file, result_file_csv)
    
    csv2nt(file=result_file_csv , ordered=False, out=result_file_nt)
        
    break

## Run N3 PP

### Procedures

In [ ]:
def runNSave(cmd, path, get_times=True):
    # result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, error = [ b.decode('UTF-8') for b in process.communicate() ]
    out = out.rstrip()
    # print("out:", out)
    # print("error:", error)
        
    with open(path, 'w') as fh:
        fh.write(out)
    
    if get_times:
        netw_time = int(re.search("networking \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        reas_time = int(re.search("reasoning \d+ \[msec cputime\] (\d+) \[msec walltime\]", error).group(1))
        return netw_time, reas_time

def record(times_file, query, data, type, phase, netw_time, reas_time):
    times_file.write(f"{query},{data},{type},{phase},{netw_time},{reas_time}\n")

### Ground & normalize

In [ ]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"
for file in os.listdir(path):    
    if not file.endswith(".sparql"):
        continue
    
    file = file[:file.index(".")] + ".n3"
    or_file = os.path.join(path, "n3", file)
    norm_file = os.path.join(path, "n3", "normalized", file)
    
    print(file)
    
    # ground
    runNSave(["eye", "--quiet", or_file, "--no-qvars", "--nope", "--pass-all"], norm_file, get_times=False)
    
    # normalize
    runNSave(["eye", "--quiet", norm_file, "aux_list.n3", "--query", "list-predicate.n3", "--quantify", "http://www.w3.org/2000/10/swap/var#", "--nope"], norm_file, get_times=False)

### Run

In [ ]:
# path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/other_systems/gmark-dominik/50/"
path = "/Users/wvw/git/n3/sparql2n3/SPARQL-to-N3/gmark_50_new/mix/"

times_file = open(os.path.join(path, "n3", "results", "times.csv"), 'w')
times_file.write("query,data,type,phase,netw_time,reas_time\n")

files = list(os.listdir(os.path.join(path, "n3", "normalized")))
files.sort()
for file in files:
    if not file.startswith("query") and not file.endswith(".n3"):
        continue
    
    name = file[:file.index(".")]
    norm_file = os.path.join(path, "n3", "normalized", file)
    data_file = "data.n3"
    data_path = os.path.join(path, data_file)
    dir_res_file = os.path.join(path, "n3", "results", "direct", file)
    bwd_res_file = os.path.join(path, "n3", "results", "bwd", file)
    fwd_res_file = os.path.join(path, "n3", "results", "fwd", file)
    
    print(file)
    
    # - direct 
    netw_time, reas_time = runNSave(["eye", data_path, "property-paths-direct.n3", "--query", norm_file, "--nope"], dir_res_file)
    record(times_file, file, data_file, 'direct', 'n/a', netw_time, reas_time)
    
    # - backward
    tmp_file = os.path.join(path, "n3", "gen", f"{name}_bwd.n3")
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation-backwards.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'bwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], bwd_res_file)
    record(times_file, file, data_file, 'bwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'bwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
        
    # - forward
    tmp_file = os.path.join(path, "n3", "gen", f"{name}_fwd.n3")
    netw_time1, reas_time1 = runNSave(["eye", norm_file, "rule-creation.n3", "--query", "rule-creation.n3", "--nope"], tmp_file)
    record(times_file, file, data_file, 'fwd', 'create', netw_time1, reas_time1)
    netw_time2, reas_time2 = runNSave(["eye", data_path, tmp_file, "--query", norm_file, "--nope"], fwd_res_file)
    record(times_file, file, data_file, 'fwd', 'run', netw_time2, reas_time2)
    record(times_file, file, data_file, 'fwd', 'total', netw_time1+netw_time2, reas_time1+reas_time2)
    
    times_file.flush()